<a href="https://colab.research.google.com/github/safuh/MyTensors/blob/main/TxtImgGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GANS Demonstrates how to generate images of handwritten digits using Deep Convulational Generative Adversial Networks(DCGAN). The code is written using the Keras Sequential API with a tf.GradientTape training loop.

Generative Adversarial Network (GANS) consists of two models that are trained simultaneously by an adversarial process. A generator learns to create images that look real, while discriminator learns to tell real images apart from fake. During training, the generator progressively becomes better at creating images that look real, while discriminator becomes better at telling them apart until an equilibrium is reached when the descriminator can no longer distinguish real from fakes.


Install dependencies

In [23]:
!pip install imageio
!pip install git+https://github.com/tensorflow/docs
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-ogfjdb0p
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-ogfjdb0p
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


import libraries

In [24]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_datasets as tfds
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
import imageio
import os
import PIL
import time
from tensorflow.keras import layers
from IPython import display

Load and prepare dataset

TFDS provides a collection of ready-to-use datasets in tf. it handles downloading and preparing the data deterministically and constructing a tf.Dataset(or np.array). TFDS is a high level wrapper around tf.data that builds efficient data pipelines


In [ ]:
tfds.list_builders()
#Construct a tf.data.Dataset
#ds = tfds.load('imagenet2012',split='train',shuffle_files=True)
ds,info = tfds.load('fashion_mnist',split='train',as_supervised=True,with_info=True,shuffle_files=True)
##fig = tfds.show_examples(ds, info)

build an input pipeline

In [93]:
#ds = ds.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)
#for image, label in ds.take(1):
  #print(image.shape, label.shape)
  #for i in range(0,len(image)):
    #print(image[i].shape,label[i])
    ##visualize data
    ##tfds.as_dataframe(ds.take(1),info)

(train_images, train_labels), (_,_)= tf.keras.datasets.fashion_mnist.load_data()

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5)/127.5# Normalize the images to [-1, 1]
BUFFER_SIZE = 60000
BATCH_SIZE = 256
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


**Build model**
The Generator
Uses Conv2DTranspose(upsampling) layers to produce an image from a seed(random noise